In [88]:
import pandas as pd
import numpy as np
from tqdm import tqdm
df = pd.read_json("C:/Users/Dileep Sathya/OneDrive/Desktop/Stock_AI_2.0/artifacts/hist_data.json")
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['TG_body_pct'] = ((df['close'] - df['open']) / df['close']) * 100

for col in ['open', 'high', 'low', 'close', 'volume']:
    df[f'prev_day_{col}'] = df.groupby('symbol')[col].shift(1)
df=df.dropna()
df=df.drop(columns=['high','low','close','volume'])


In [89]:
df['prev_range_pct'] = ((df['prev_day_high'] - df['prev_day_low']) / df['prev_day_high']) * 100
    
    # Daily body range % (Close-Open relative to previous close)
df['prev_body_pct'] = ((df['prev_day_close'] - df['prev_day_open']) / df['prev_day_close']) * 100
def calculate_atr(df, window=14):
    hl = df['prev_day_high'] - df['prev_day_low']
    hc = abs(df['prev_day_high'] - df['prev_day_close'])
    lc = abs(df['prev_day_low'] - df['prev_day_close'])
    tr = pd.concat([hl, hc, lc], axis=1).max(axis=1)
    return tr.rolling(window).mean()
df['atr_14'] = df.groupby('symbol', group_keys=False).apply(calculate_atr)
df['atr_pct'] = (df['atr_14'] / df['prev_day_close']) * 100  # ATR as % of price
df['gap_pct'] = ((df['open'] - df['prev_day_close']) / df['prev_day_close']) * 100

C:\Users\Dileep Sathya\AppData\Local\Temp\ipykernel_14924\135472878.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['atr_14'] = df.groupby('symbol', group_keys=False).apply(calculate_atr)


In [90]:
def compute_ema_features(df, spans):
    for span in spans:
        ema_col = f'ema_{span}'
        df[ema_col] = df.groupby('symbol')['prev_day_close'].transform(lambda x: x.ewm(span=span, adjust=False).mean())
        df[f'open_ema_{span}_dist_pct'] = ((df['open'] - df[ema_col]) / df[ema_col]) * 100
    return df

df = compute_ema_features(df, [5, 20, 50])

In [91]:
df['avg_volume_10'] = df.groupby('symbol')['prev_day_volume'].transform(lambda x: x.rolling(10).mean())
df['volume_pct_change'] = np.where(
    df['avg_volume_10'] != 0,
    (df['prev_day_volume'] - df['avg_volume_10']) / df['avg_volume_10'] * 100,
    0
)


In [92]:
df=df.dropna()

In [93]:
df['final_tg'] = 'no_trade'  # default
df.loc[df['TG_body_pct'] > 2, 'final_tg'] = 'buy'
df.loc[df['TG_body_pct'] < -2, 'final_tg'] = 'sell'
df=df.drop(columns=['TG_body_pct'])
df

,date,open,symbol,day_of_week,month,prev_day_open,prev_day_high,prev_day_low,prev_day_close,prev_day_volume,...,gap_pct,ema_5,open_ema_5_dist_pct,ema_20,open_ema_20_dist_pct,ema_50,open_ema_50_dist_pct,avg_volume_10,volume_pct_change,final_tg
14,2000-01-21,14.40,BHEL,4,1,15.00,15.00,14.37,14.41,157434.0,...,-0.069396,14.415881,-0.110164,14.532127,-0.909209,14.715263,-2.142423,337679.0,-53.377616,no_trade
15,2000-01-24,14.37,BHEL,0,1,14.40,14.46,14.22,14.32,256471.0,...,0.349162,14.383921,-0.096780,14.511925,-0.977987,14.699763,-2.243320,323107.3,-20.623582,sell
16,2000-01-25,13.73,BHEL,1,1,14.37,14.40,13.73,13.84,205997.0,...,-0.794798,14.202614,-3.327654,14.447932,-4.969098,14.666046,-6.382405,306846.7,-32.866477,no_trade
17,2000-01-27,13.86,BHEL,3,1,13.73,13.86,13.47,13.53,307686.0,...,2.439024,13.978409,-0.847086,14.360510,-3.485321,14.621496,-5.208056,306651.9,0.337223,sell
18,2000-01-28,13.07,BHEL,4,1,13.86,13.93,13.15,13.30,325289.0,...,-1.729323,13.752273,-4.961164,14.259509,-8.341865,14.569672,-10.293109,319403.7,1.842590,sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285163,2025-06-09,617.00,SUNTV,0,6,620.00,622.65,610.60,614.85,163218.0,...,0.349679,624.030611,-1.126645,629.313687,-1.956685,630.378175,-2.122246,123830.3,31.807805,no_trade
285164,2025-06-10,635.05,SUNTV,1,6,617.00,627.95,616.00,626.05,82259.0,...,1.437585,624.703740,1.656187,629.002860,0.961385,630.208443,0.768247,125800.6,-34.611600,no_trade
285165,2025-06-11,630.00,SUNTV,2,6,635.05,649.90,623.10,627.05,1027524.0,...,0.470457,625.485827,0.721707,628.816873,0.188151,630.084582,-0.013424,222128.2,362.581518,no_trade
285166,2025-06-12,625.00,SUNTV,3,6,630.00,630.50,621.10,624.65,184722.0,...,0.056031,625.207218,-0.033144,628.420028,-0.544226,629.871461,-0.773406,230446.7,-19.841768,no_trade


In [94]:
df=df.drop(columns=['symbol','open','atr_14','prev_day_open','prev_day_high','prev_day_low','prev_day_close','prev_day_volume','ema_5','ema_20','ema_50','avg_volume_10'])


In [95]:
df

,date,day_of_week,month,prev_range_pct,prev_body_pct,atr_pct,gap_pct,open_ema_5_dist_pct,open_ema_20_dist_pct,open_ema_50_dist_pct,volume_pct_change,final_tg
14,2000-01-21,4,1,4.200000,-4.094379,5.863983,-0.069396,-0.110164,-0.909209,-2.142423,-53.377616,no_trade
15,2000-01-24,0,1,1.659751,-0.558659,5.771149,0.349162,-0.096780,-0.977987,-2.243320,-20.623582,sell
16,2000-01-25,1,1,4.652778,-3.829480,5.868084,-0.794798,-3.327654,-4.969098,-6.382405,-32.866477,no_trade
17,2000-01-27,3,1,2.813853,-1.478197,5.611868,2.439024,-0.847086,-3.485321,-5.208056,0.337223,sell
18,2000-01-28,4,1,5.599426,-4.210526,5.601504,-1.729323,-4.961164,-8.341865,-10.293109,1.842590,sell
...,...,...,...,...,...,...,...,...,...,...,...,...
285163,2025-06-09,0,6,1.935277,-0.837603,2.507580,0.349679,-1.126645,-1.956685,-2.122246,31.807805,no_trade
285164,2025-06-10,1,6,1.903018,1.445571,2.383424,1.437585,1.656187,0.961385,0.768247,-34.611600,no_trade
285165,2025-06-11,2,6,4.123711,-1.275815,2.498662,0.470457,0.721707,0.188151,-0.013424,362.581518,no_trade
285166,2025-06-12,3,6,1.490880,-0.856480,2.444226,0.056031,-0.033144,-0.544226,-0.773406,-19.841768,no_trade
